# Generate synthetic spectra using ramanchada2

In [ ]:
import ramanchada2 as rc2
import ipywidgets as ipyw
import numpy as np
import matplotlib.pyplot as plt

spe_tmp = {'orig': rc2.spectrum.from_delta_lines(deltas={500:1e3, 900:1.5e3})}
def convolve(lineshape, sigma):
    spe_tmp['convolved'] = spe_tmp['orig'].convolve(lineshape=lineshape, sigma=sigma)
    spe_tmp['convolved'].plot()
def add_baseline(n_freq, amplitude, pedestal, rng_seed, a1, a2):
    spe_tmp['baseline'] = spe_tmp['convolved'].add_baseline(n_freq=n_freq, amplitude=amplitude, pedestal=pedestal, rng_seed=rng_seed, func=lambda x: x*a1 + x**2*a2)
    spe_tmp['baseline'].plot()
def add_noise(scale, rng_seed):
    spe_tmp['noise'] = spe_tmp['baseline'].add_poisson_noise(scale=scale, rng_seed=rng_seed)
    spe_tmp['noise'].plot()

## end-to-end example

In [ ]:
for sigma in [10, 20, 30]:
    spe = rc2.spectrum.from_delta_lines(deltas={500:1e3, 700:1.5e3})
    spe = spe.convolve(lineshape='voigt', sigma=sigma, gamma=3)
    spe = spe.add_baseline(n_freq=50, amplitude=10, pedestal=10, func=lambda x: x*.006 + x**2*.00001)
    spe = spe.add_poisson_noise(scale=.01)
    spe.plot()

## documentation of the methods

In [ ]:
help(rc2.spectrum.from_delta_lines)

In [ ]:
help(spe.convolve)

In [ ]:
help(spe.add_baseline)

In [ ]:
help(spe.add_poisson_noise)

## Interactive example

In [ ]:
ipyw.interact(
    convolve,
    lineshape=ipyw.widgets.Dropdown(options=['voigt', 'gaussian', 'lorentzian', 'pvoigt', 'moffat']),
    sigma=ipyw.widgets.FloatSlider(min=1, max=50, value=5),
)
None

In [ ]:
ipyw.interact(
    add_baseline,
    n_freq=ipyw.widgets.IntSlider(min=3, max=500, value=50),
    a1=ipyw.widgets.FloatSlider(min=-.2, max=.2, value=.0, step=.00001),
    a2=ipyw.widgets.FloatSlider(min=-.0002, max=.0002, value=.0, step=.00001,  readout_format='.5f'),
    amplitude=ipyw.widgets.FloatSlider(min=0, max=50, value=10),
    pedestal=ipyw.widgets.FloatSlider(min=1, max=50, value=10),
    rng_seed=ipyw.widgets.IntSlider(min=0, max=999999, value=0)
)
None

In [ ]:
ipyw.interact(
    add_noise,
    scale=ipyw.widgets.FloatSlider(min=0, max=2, value=.01),
    rng_seed=ipyw.widgets.IntSlider(min=0, max=999999, value=0)
)
None

In [ ]:
import ramanchada2 as rc2
spe = rc2.spectrum.from_delta_lines(deltas={500:1e3, 700:1.5e3})
spe = spe.convolve(lineshape='voigt', sigma=5)
spe = spe.add_baseline(n_freq=50, amplitude=10, pedestal=10)
spe = spe.add_poisson_noise(scale=.01)
spe.plot()

## convolution usage

In [ ]:
spe = rc2.spectrum.from_delta_lines(deltas={500:1e3, 700:1.5e3})
spe.convolve(lineshape='voigt', sigma=3, gamma=5).plot(label=r'voigt$(\sigma=3, \gamma=5)$')
spe.convolve(lineshape=np.array([.1]*15+[.4]*15+[1]*15+[0]+[1]*25+[.4]*25+[.1]*25)/23).plot(label='user-defined array')
spe.convolve(lineshape=lambda x, w: np.exp(-(x/w)**2)/23, w=20).plot(label='user-defined gaussian')